# Convert GEDI to COPC

1. Switch to the PDAL conda env kernel `pdal24`
1. Execute the pipeline to convert

TODOs: Include more attributes from the source format.

Need an env with newer pdal

Copy and Paste these to terminal
```
conda install -y -c conda-forge mamba
mamba create -n pdal24 -c conda-forge pdal ipykernel pyproj gdal python-pdal
conda activate pdal24
python3 -m ipykernel install --user
```

In [1]:
# %pip install -U pystac-client
# %pip install -U boto3 botocore
# %pip install -U pyproj

  Using cached pystac_client-0.7.5-py3-none-any.whl.metadata (5.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached pystac-1.9.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached jsonschema-4.20.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached jsonschema_specifications-2023.11.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.32.0-py3-none-any.whl.metadata (2.7 kB)
Using cached pystac_client-0.7.5-py3-none-any.whl (33 kB)
Using cached pystac-1.9.0-py3-none-any.whl (181 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
Using cached idn

In [1]:
from pystac_client import Client
URL = 'https://stac.maap-project.org/'
cat = Client.open(URL)
stac_collections = list(cat.get_collections())
stac_collections

[<CollectionClient id=Landsat8_SurfaceReflectance>,
 <CollectionClient id=Global_PALSAR2_PALSAR_FNF>,
 <CollectionClient id=Global_Forest_Change_2000-2017>,
 <CollectionClient id=AFRISAR_DLR2>,
 <CollectionClient id=GlobCover_09>,
 <CollectionClient id=AfriSAR_UAVSAR_KZ>,
 <CollectionClient id=AfriSAR_UAVSAR_Ungeocoded_Covariance>,
 <CollectionClient id=AfriSAR_UAVSAR_Normalization_Area>,
 <CollectionClient id=AfriSAR_UAVSAR_Geocoded_SLC>,
 <CollectionClient id=AfriSAR_UAVSAR_Geocoded_Covariance>,
 <CollectionClient id=GlobCover_05_06>,
 <CollectionClient id=GEDI_CalVal_Field_Data>,
 <CollectionClient id=AfriSAR_UAVSAR_Coreg_SLC>,
 <CollectionClient id=GEDI_CalVal_Lidar_Data_Compressed>,
 <CollectionClient id=ABoVE_UAVSAR_PALSAR>,
 <CollectionClient id=AFRISAR_DLR>,
 <CollectionClient id=BIOSAR1>,
 <CollectionClient id=GEDI_CalVal_Lidar_Data>,
 <CollectionClient id=icesat2-boreal>,
 <CollectionClient id=ICESat2_Boreal_AGB_tindex_average>,
 <CollectionClient id=NCEO_Africa_AGB_100m_2017

In [2]:
collection = cat.get_collection("GEDI_CalVal_Lidar_Data_Compressed")
collection

<CollectionClient id=GEDI_CalVal_Lidar_Data_Compressed>

In [3]:
collection_items = list(cat.search(collections=[collection.id], max_items=10).items())
collection_items

[<Item id=usa_moscowmtn_MM03_1401>]

In [4]:
import os
dataDir = "./data"

# check if directory exists -> if directory doesn't exist, directory is created
if not os.path.exists(dataDir):
    os.mkdir(dataDir)

In [5]:
item_asset = collection_items[0].assets['data'].href
item_asset
# item_asset = item.href
!wget -P {dataDir} {item_asset}

s3://nasa-maap-data-store/file-staging/nasa-map/GEDI_CalVal_Lidar_Data_Compressed___2/usa_moscowmtn_MM03_1401.laz: Unsupported scheme ‘s3’.


In [6]:
import boto3
import botocore

BUCKET_NAME = 'nasa-maap-data-store' # replace with your bucket name
KEY = 'file-staging/nasa-map/GEDI_CalVal_Lidar_Data___2/usa_neonsrer_2019_NEON_D14_SRER_DP1_L090-1_2019091314_unclassified_point_cloud_0000004.las' # replace with your object key
fname = 'usa_neonsrer_2019_NEON_D14_SRER_DP1_L090-1_2019091314_unclassified_point_cloud_0000004.las'
# KEY = 'file-staging/nasa-map/GEDI_CalVal_Lidar_Data_Compressed___2/usa_moscowmtn_MM03_1401.laz'
s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, f'data/{fname}')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [7]:
import pdal
import pyproj

pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed
pj_obj_create: Open of /opt/conda/envs/pdal24/share/proj failed


In [8]:
las_file = f'data/{fname}'
ofname = fname.split('.')[0]+'.copc.laz'
copc_file = f'data/{ofname}'

In [9]:
json = """{
  "pipeline": [
    {
        "filename":las_file,
        "type":"readers.las"
    },
    {
        "type":"writers.copc",
        "filename":copc_file
    }
]
}"""

pipeline = pdal.Reader.las(filename=las_file) | pdal.Writer.copc(filename=copc_file)
print(pipeline.execute())

1


proj_create_from_database: Open of /opt/conda/envs/pdal24/share/proj failed


In [10]:
arrays = pipeline.arrays
metadata = pipeline.metadata
log = pipeline.log

In [11]:
print(metadata)

{'metadata': {'readers.las': {'comp_spatialreference': 'COMPD_CS["WGS 84 / UTM zone 12N + unknown",PROJCS["WGS 84 / UTM zone 12N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]],VERT_CS["",VERT_DATUM["unknown",2005],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Up",UP]]]', 'compressed': False, 'copc': False, 'count': 1, 'creation_doy': 293, 'creation_year': 2019, 'dataformat_id': 1, 'dataoffset': 329, 'filesource_id': 9011, 'global_encoding': 0, 'global_encoding_base64': 'AAA=', 'gtiff': 'Geotiff_Information:\n   Version: 1\n   Key_Revision: 1.0\n   Tagged_Information:\n      End

In [12]:
!ls -lh {os.path.dirname(copc_file)}

total 228K
-rw-r--r-- 1 root root 1.6K Dec  8 14:53 test.copc.laz
-rw-r--r-- 1 root root  357 Dec  8 14:51 test.las
-rw-r--r-- 1 root root 1.6K Dec  8 14:42 test.laz
-rw-r--r-- 1 root root 205K Nov 30 14:29 test2.laz
-rw-r--r-- 1 root root 1.6K Dec 19 14:05 usa_neonsrer_2019_NEON_D14_SRER_DP1_L090-1_2019091314_unclassified_point_cloud_0000004.copc.laz
-rw-r--r-- 1 root root  357 Dec 19 14:05 usa_neonsrer_2019_NEON_D14_SRER_DP1_L090-1_2019091314_unclassified_point_cloud_0000004.las
